In [35]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

In [2]:
runs = [282, 283, 284, 285, 287]


fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname)

for i in range(1,len(runs)):

    topics1 = DynamicTopic.objects.filter(run_id=runs[i-1])
    topics2 = DynamicTopic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [3]:
runs = list(range(295,300))

writer = pd.ExcelWriter(fname)
fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [49]:
runs = list(range(308,318))


#runs = [295,296,297]

col1s = []
col2s = []
ss = []
scols = []

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])
    

    df = pd.DataFrame.from_dict(list(topics2.values('title','score')))
    
    df2 = pd.DataFrame.from_dict([{'title': 'None','score': 0}])
    
    df = df.append(df2)

    
    
    col1 = "run_{}_topics_{}".format(runs[i-1],topics1.count())
    
    scol = "scores_{}".format(runs[i])
    bscol = "scores_{}".format(runs[i-1])
    
    if i==1:
        scols.append(bscol)
    
    col1s.append(col1)
    scols.append(scol)
    
    col2 = "run_{}_topics_{}".format(runs[i], topics2.count())
    
    col2s.append(col2) 
    
    s = "similarity_{}-{}".format(runs[i-1],runs[i])
    ss.append(s)
    
    df = df.rename(columns = {'title': col2, 'score': scol})

    df[s] = 0
    df[col1] = "None"
    df[bscol] = 0

    for t in topics2:
        scores = [0]
        titles = [""]
        tscores = [0]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)
                tscores.append(ct.score)

        
        m = max(scores)
        df.loc[df[col2]==t.title, s] = m
        if m==0:
            df.loc[df[col2]==t.title, col1] = 'None'
        else:
            df.loc[df[col2]==t.title, col1] = titles[scores.index(max(scores))]
            df.loc[df[col2]==t.title, bscol] = tscores[scores.index(max(scores))]
            

    if i==1:   
        #df = pd.DataFrame.from_dict(list(topics2.values('title')))
        mdf = df
    else:
        mdf = mdf.merge(df,how="outer").fillna("")
    

columns = []
for i in range(len(col1s)):
    columns.append(col1s[i])
    columns.append(scols[i])
    columns.append(ss[i])
    if i == len(col1s)-1:
        columns.append(col2s[i])
        columns.append(scols[i+1])

print(columns)

mdf = mdf[columns]

mdf.head()



['run_308_topics_30', 'scores_308', 'similarity_308-309', 'run_309_topics_35', 'scores_309', 'similarity_309-310', 'run_310_topics_40', 'scores_310', 'similarity_310-311', 'run_311_topics_45', 'scores_311', 'similarity_311-312', 'run_312_topics_50', 'scores_312', 'similarity_312-313', 'run_313_topics_55', 'scores_313', 'similarity_313-314', 'run_314_topics_60', 'scores_314', 'similarity_314-315', 'run_315_topics_65', 'scores_315', 'similarity_315-316', 'run_316_topics_70', 'scores_316', 'similarity_316-317', 'run_317_topics_90', 'scores_317']


,run_308_topics_30,scores_308,similarity_308-309,run_309_topics_35,scores_309,similarity_309-310,run_310_topics_40,scores_310,similarity_310-311,run_311_topics_45,...,scores_314,similarity_314-315,run_315_topics_65,scores_315,similarity_315-316,run_316_topics_70,scores_316,similarity_316-317,run_317_topics_90,scores_317
0,"{model, simul, use}",35.063653,3,"{indic, method, assess}",20.444094,9,"{indic, assess, method}",15.995,9,"{indic, method, assess}",...,24.0792,9,"{indic, method, assess}",20.1381,9,"{indic, assess, method}",17.9837,6,"{indic, index, measur}",12.1295
1,"{model, simul, use}",35.063653,3,"{indic, method, assess}",20.444094,9,"{indic, assess, method}",15.995,9,"{indic, method, assess}",...,24.0792,9,"{indic, method, assess}",20.1381,9,"{indic, assess, method}",17.9837,6,"{assess, method, analysi}",30.8486
2,"{energi, renew, consumpt}",29.331418,10,"{energi, renew, consumpt}",27.496250,10,"{energi, renew, consumpt}",26.4508,10,"{energi, renew, consumpt}",...,22.5572,10,"{energi, renew, sourc}",21.6333,10,"{energi, renew, sourc}",20.826,2,"{biomass, potenti, residu}",14.6792
3,"{energi, renew, consumpt}",29.331418,10,"{energi, renew, consumpt}",27.496250,10,"{energi, renew, consumpt}",26.4508,10,"{energi, renew, consumpt}",...,22.5572,10,"{energi, renew, sourc}",21.6333,10,"{energi, renew, sourc}",20.826,7,"{energi, renew, sourc}",17.8682
4,"{water, irrig, resourc}",37.277328,9,"{water, irrig, resourc}",35.562412,9,"{water, river, suppli}",32.4825,10,"{water, river, suppli}",...,28.1007,6,"{water, river, suppli}",27.7967,6,"{water, suppli, demand}",26.1472,9,"{water, suppli, demand}",23.992


In [53]:
#col1s.append(col2)


#mdf = mdf.sort_values(columns,ascending=False).reset_index()


res = mdf.groupby(columns)



res = res.apply(lambda x: x.sort_values(s,ascending=False))

l = len(res)




fname = "../../data/topic_data/run_compare_{}_{}.xlsx".format(runs[0],runs[len(runs)-1])

writer = pd.ExcelWriter(fname, engine='xlsxwriter')

res.to_excel(writer)

worksheet = writer.sheets['Sheet1']

for i in range(len(ss)):
    if (i+1)*3 > 26:
        c = 'A'+chr(ord('A')-1+((i+1)*3)-26)
    else:
        c = chr(ord('A')-1+(i+1)*3)
    r = "{}2:{}{}".format(c,c,len(res))
    print(r)
    
    worksheet.conditional_format(r, {
        'type': '3_color_scale',
        'min_value': 0,
        'mid_value': 5,
        'max_value': 10,
        'min_type': num,
        'mid_type': num,
        'max_type': num,
    })

writer.save()

res.head() 
    
    
res.head()

C2:C108
F2:F108
I2:I108
L2:L108
O2:O108
R2:R108
U2:U108
X2:X108
AA2:AA108


run_308_topics_30  \
run_308_topics_30           scores_308 similarity_308-309 run_309_topics_35          scores_309 similarity_309-310 run_310_topics_40          scores_310 similarity_310-311 run_311_topics_45          scores_311 similarity_311-312 run_312_topics_50          scores_312 similarity_312-313 run_313_topics_55           scores_313 similarity_313-314 run_314_topics_60          scores_314 similarity_314-315 run_315_topics_65         scores_315       similarity_315-316 run_316_topics_70         scores_316       similarity_316-317 run_317_topics_90             scores_317                                          
None                        0.000000   0                  None                       0.000000   0.0                None                       0.000000   0.0                None                       0.000000   0.0                None                       0.000000   0.0                None                        0.000000   0.0                None                       0.000000   0.0                None                      0.0              0.0                None                      0.0              0.0                None                          0.0              107                         None   
{catalyst, oxid, reaction}  31.183143  9                  {catalyst, oxid, reaction} 23.574559  9.0                {catalyst, oxid, reaction} 13.858472  9.0                {catalyst, oxid, reaction} 17.767461  10.0               {catalyst, oxid, reaction} 14.203130  10.0               {catalyst, oxid, reaction}  10.257899  10.0               {catalyst, oxid, reaction} 8.403417   9.0                {catalyst, oxid, solvent} 6.69887609238191 10.0               {catalyst, oxid, solvent} 6.00980881678852 4.0                {catalyst, reaction, catalyt} 2.99273377449593 70    {catalyst, oxid, reaction}   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          7.0                {oxid, solvent, format}       15.1817497537324 71    {catalyst, oxid, reaction}   
{chang, climat, adapt}      31.611345  9                  {chang, climat, adapt}     28.226901  10.0               {chang, climat, adapt}     27.385216  9.0                {chang, climat, adapt}     22.488560  9.0                {chang, climat, adapt}     23.092477  10.0               {chang, climat, adapt}      19.675137  10.0               {chang, climat, adapt}     17.050416  8.0                {chang, climat, adapt}    16.6344833525782 9.0                {chang, climat, adapt}    14.6474247350682 9.0                {chang, climat, adapt}        12.5533590179524 89        {chang, climat, adapt}   
{communiti, local, resourc} 42.406726  10                 {communiti, local, govern} 35.395750  9.0                {communiti, local, govern} 29.385899  9.0                {communiti, local, rural}  10.154019  7.0                {communiti, local, peopl}  16.924496  9.0                {communiti, local, monitor} 14.507632  3.0                {monitor, data, wildlif}   21.472756                                                                                                                                                                                                19   {communiti, local, resourc}   

                                                                                                                                                                                                                                                                                                                                          

In [ ]:
print(mdf)

In [ ]:
runs = list(range(295,300))

runs = []

for i in range(len(runs)):

    topics = Topic.objects.filter(run_id=runs[i])
    
    tree = []
    
    for t in topics:
        
        tdict = {
            "parent_similarity": 0,
            "run": runs[i],
            "ntopics": topics.count(),
            "title": t.title,
            "sim_topics": [],
            "child_topics": 0
        }
        
        if i==0:
            tree.append(tdict)
        else:
            
            scores = [0]
            titles = [""]
            for ct in comparison_topics:
                score = len(set(t.top_words).intersection(set(ct.top_words)))
                if score>0:
                    scores.append(score)
                    titles.append(ct.title)


                parent = titles[scores.index(max(scores))]
                
                
                
                print(m)

In [ ]:
def find_similar(x):
    return(1,2)

df['bla'], df['x'] = zip(df.map(find_similar))
                         
df.head()